In [ ]:
from pathlib import Path
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import pdb
import pickle
import numpy as np
from typing_extensions import Annotated
import time
import copy

from mbnpy import model, config, trans, variable, brc, branch, cpm, operation

HOME = Path().resolve()

%matplotlib inline

In [9]:
#file_model=r'./BNS_JT/demos/Wheat/model_Wooroloo-Merredin_5.pk'
#file_dmg=r'./BNS_JT/demos/Wheat/avg_gmf_69_dmg.csv'

file_model=r'./model_Wooroloo-Merredin_5.pk'
file_dmg=r'./avg_gmf_69_dmg.csv'

key = 'Wooroloo-Merredin'

In [10]:
cfg = config.Config(HOME.joinpath('./config.json'))

with open(file_model, 'rb') as f:
    dump = pickle.load(f)
    cpms = dump['cpms']
    varis = dump['varis']
    path_names = dump['path_names']

# assign cpms given scenario
probs = pd.read_csv(file_dmg, index_col=0)

probs.index = probs.index.astype('str')
probs = probs.to_dict('index')

for k, v in cfg.infra['nodes'].items():
    #varis[k] = variable.Variable(name=k, values = [np.inf, 1.0*v['weight']])
    try:
        pf = probs[k]['Extensive'] + probs[k]['Complete']
    except KeyError:
        pf = 0.0
    finally:
        cpms[k] = cpm.Cpm(variables = [varis[k]], no_child=1,
                        C = np.array([0, 1]).T, p = [pf, 1-pf])

od_pair = cfg.infra['ODs'][key]
od_name = '_'.join(od_pair)
VE_ord = list(cfg.infra['nodes'].keys()) + path_names
vars_inf = operation.get_inf_vars(cpms, od_name, VE_ord)

scenario to be added later


In [15]:
print(cpms[od_name])

od_name_debug = od_name + '_debug'
cpms[od_name_debug] = copy.deepcopy( cpms[od_name] )

Cpm(
    variables=['Wooroloo_Merredin', 'Wooroloo_Merredin_0', 'Wooroloo_Merredin_1', 'Wooroloo_Merredin_2', 'Wooroloo_Merredin_3', 'Wooroloo_Merredin_4'],
    no_child=1,
    C=[[5 1 2 2 2 2]
 [4 0 1 2 2 2]
 [3 0 0 1 2 2]
 [2 0 0 0 1 2]
 [1 0 0 0 0 1]
 [0 0 0 0 0 0]],
    p=[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]],
)


In [30]:
cols_to_keep = [0, 1, 2, 3]

cpms[od_name_debug].variables = [cpms[od_name].variables[i] for i in cols_to_keep]
varis[od_name_debug] = variable.Variable(name=od_name_debug, values = [np.inf] + [varis[od_name].values[cols_to_keep[i]] for i in cols_to_keep[1:]])
cpms[od_name_debug].C = np.array([[3, 1, 2, 2], [2, 0, 1, 2], [1, 0, 0, 1], [0, 0, 0, 0]])
cpms[od_name_debug].p = np.array([1.0, 1.0, 1.0, 1.0])



In [31]:
print(cpms[od_name_debug])
print(varis[od_name_debug])

Cpm(
    variables=['Wooroloo_Merredin', 'Wooroloo_Merredin_0', 'Wooroloo_Merredin_1', 'Wooroloo_Merredin_2'],
    no_child=1,
    C=[[3 1 2 2]
 [2 0 1 2]
 [1 0 0 1]
 [0 0 0 0]],
    p=[[1.]
 [1.]
 [1.]
 [1.]],
)
'Variable(name=Wooroloo_Merredin_debug, values=[inf, 8485, 8484, 8441], B=[{0}, {1}, {2}, {3}, {0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}, {0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}, {0, 1, 2, 3}])'


In [32]:
VE_ord = list(cfg.infra['nodes'].keys()) + path_names
vars_inf = operation.get_inf_vars(cpms, od_name_debug, VE_ord)
print(vars_inf)

['Kellerberrin', 'Meckering', 'Cunderdin', '0612', '0613', '1047', '1006', '1007', '1050', '0627', 'Merredin', '1149', '9186', 'Wooroloo', '0606', '9306', '1046', '1468', '1504', '1467', '1466', '1465', '1464', '1463', '9187', '1150', '1153', '1154', '0617', 'Wooroloo_Merredin_0', 'Wooroloo_Merredin_1', 'Wooroloo_Merredin_2', 'Wooroloo_Merredin_debug']


In [33]:
Mod_debug = operation.variable_elim([cpms[k] for k in vars_inf], [v for v in vars_inf if v != od_name_debug])

In [34]:
print(sum(Mod_debug.p))
print(Mod_debug)

[1.]
Cpm(
    variables=['Wooroloo_Merredin'],
    no_child=1,
    C=[[0]
 [3]
 [2]],
    p=[[0.42887808]
 [0.51694973]
 [0.05417218]],
)


In [23]:
print(cpms[od_name])

Cpm(
    variables=['Wooroloo_Merredin', 'Wooroloo_Merredin_0', 'Wooroloo_Merredin_1', 'Wooroloo_Merredin_2', 'Wooroloo_Merredin_3', 'Wooroloo_Merredin_4'],
    no_child=1,
    C=[[5 1 2 2 2 2]
 [4 0 1 2 2 2]
 [3 0 0 1 2 2]
 [2 0 0 0 1 2]
 [1 0 0 0 0 1]
 [0 0 0 0 0 0]],
    p=[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]],
)


In [37]:
print([v.name for v in cpms[od_name + '_0'].variables])
print([v.name for v in cpms[od_name + '_1'].variables])
print([v.name for v in cpms[od_name + '_2'].variables])

['Wooroloo_Merredin_0', 'Wooroloo', '0606', '9306', '1046', '1047', '9186', '1468', '1504', '1467', '1466', '1465', '1464', '1463', '9187', '1149', '1150', '1153', '1154', '0617', 'Meckering', 'Cunderdin', 'Kellerberrin', '1006', '1007', '1050', '0627', 'Merredin']
['Wooroloo_Merredin_1', 'Wooroloo', '0606', '9306', '1046', '1047', '0612', '0613', '1149', '1150', '1153', '1154', '0617', 'Meckering', 'Cunderdin', 'Kellerberrin', '1006', '1007', '1050', '0627', 'Merredin']
['Wooroloo_Merredin_2', 'Wooroloo', '0606', '9306', '1046', '1047', '9186', '0612', '0613', '1149', '1150', '1153', '1154', '0617', 'Meckering', 'Cunderdin', 'Kellerberrin', '1006', '1007', '1050', '0627', 'Merredin']
